In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
# multiclass example with PS, SIRO, FCFS, HOL priority
model = Network('MyNetwork')

# Block 1: nodes
node = np.empty(6, dtype=object)
node[0] = Source(model, 'Source')
node[1] = Queue(model, 'WebServer', SchedStrategy.FCFS)
node[2] = Queue(model, 'Storage1', SchedStrategy.SIRO)
node[3] = Queue(model, 'Storage2', SchedStrategy.PS)
node[4] = Queue(model, 'Storage3', SchedStrategy.HOL)
node[5] = Sink(model, 'Sink')

# Block 2: classes
jobclass = np.empty(3, dtype=object)
jobclass[0] = OpenClass(model, 'Class1', 0)
jobclass[1] = OpenClass(model, 'Class2', 1) # lower priority (priority 1 < priority 0)
jobclass[2] = OpenClass(model, 'Class3', 0)

node[0].set_arrival(jobclass[0], Exp.fit_mean(10.0)) # (Source of Customers,Class0)
node[0].set_arrival(jobclass[1], Exp.fit_mean(10.0)) # (Source of Customers,Class1)
node[0].set_arrival(jobclass[2], Exp.fit_mean(10.0)) # (Source of Customers,Class2)
node[1].set_service(jobclass[0], Exp.fit_mean(0.3)) # (WebServer,Class0)
node[1].set_service(jobclass[1], Exp.fit_mean(0.5)) # (WebServer,Class1)
node[1].set_service(jobclass[2], Exp.fit_mean(0.6)) # (WebServer,Class2)
node[2].set_service(jobclass[0], Exp.fit_mean(1.1)) # (Storage1,Class0)
node[2].set_service(jobclass[1], Exp.fit_mean(1.3)) # (Storage1,Class1)
node[2].set_service(jobclass[2], Exp.fit_mean(1.5)) # (Storage1,Class2)
node[3].set_service(jobclass[0], Exp.fit_mean(2.0)) # (Storage2,Class0)
node[3].set_service(jobclass[1], Exp.fit_mean(2.1)) # (Storage2,Class1)
node[3].set_service(jobclass[2], Exp.fit_mean(1.9)) # (Storage2,Class2)
node[4].set_service(jobclass[0], Exp.fit_mean(2.5)) # (Storage3,Class0)
node[4].set_service(jobclass[1], Exp.fit_mean(1.9)) # (Storage3,Class1)
node[4].set_service(jobclass[2], Exp.fit_mean(4.3)) # (Storage3,Class2)

# Block 3: topology
P = model.init_routing_matrix() # initialize routing matrix
P.set(jobclass[0],jobclass[0],node[0],node[1], 1.0) # (Source of Customers,Class0) -> (WebServer,Class0)
P.set(jobclass[0],jobclass[0],node[1],node[2], 2.50e-01) # (WebServer,Class0) -> (Storage1,Class0)
P.set(jobclass[0],jobclass[0],node[1],node[3], 2.50e-01) # (WebServer,Class0) -> (Storage2,Class0)
P.set(jobclass[0],jobclass[0],node[1],node[4], 2.50e-01) # (WebServer,Class0) -> (Storage3,Class0)
P.set(jobclass[0],jobclass[0],node[1],node[5], 2.50e-01) # (WebServer,Class0) -> (Out,Class0)
P.set(jobclass[0],jobclass[0],node[2],node[1], 1.0) # (Storage1,Class0) -> (WebServer,Class0)
P.set(jobclass[0],jobclass[0],node[3],node[1], 1.0) # (Storage2,Class0) -> (WebServer,Class0)
P.set(jobclass[0],jobclass[0],node[4],node[1], 1.0) # (Storage3,Class0) -> (WebServer,Class0)
P.set(jobclass[1],jobclass[1],node[0],node[1], 1.0) # (Source of Customers,Class1) -> (WebServer,Class1)
P.set(jobclass[1],jobclass[1],node[1],node[2],2.50e-01) # (WebServer,Class1) -> (Storage1,Class1)
P.set(jobclass[1],jobclass[1],node[1],node[3],2.50e-01) # (WebServer,Class1) -> (Storage2,Class1)
P.set(jobclass[1],jobclass[1],node[1],node[4],2.50e-01) # (WebServer,Class1) -> (Storage3,Class1)
P.set(jobclass[1],jobclass[1],node[1],node[5],2.50e-01) # (WebServer,Class1) -> (Out,Class1)
P.set(jobclass[1],jobclass[1],node[2],node[1],1.0) # (Storage1,Class1) -> (WebServer,Class1)
P.set(jobclass[1],jobclass[1],node[3],node[1],1.0) # (Storage2,Class1) -> (WebServer,Class1)
P.set(jobclass[1],jobclass[1],node[4],node[1],1.0) # (Storage3,Class1) -> (WebServer,Class1)
P.set(jobclass[2],jobclass[2],node[0],node[1],1.0) # (Source of Customers,Class2) -> (WebServer,Class2)
P.set(jobclass[2],jobclass[2],node[1],node[2],2.50e-01) # (WebServer,Class2) -> (Storage1,Class2)
P.set(jobclass[2],jobclass[2],node[1],node[3],2.50e-01) # (WebServer,Class2) -> (Storage2,Class2)
P.set(jobclass[2],jobclass[2],node[1],node[4],2.50e-01) # (WebServer,Class2) -> (Storage3,Class2)
P.set(jobclass[2],jobclass[2],node[1],node[5],2.50e-01) # (WebServer,Class2) -> (Out,Class2)
P.set(jobclass[2],jobclass[2],node[2],node[1],1.0) # (Storage1,Class2) -> (WebServer,Class2)
P.set(jobclass[2],jobclass[2],node[3],node[1],1.0) # (Storage2,Class2) -> (WebServer,Class2)
P.set(jobclass[2],jobclass[2],node[4],node[1],1.0) # (Storage3,Class2) -> (WebServer,Class2)
model.link(P)

In [ ]:
# CTMC with cutoff=1 to match MATLAB
solver = CTMC(model, seed=23000, cutoff=1, verbose=VerboseLevel.SILENT)
AvgTable = solver.avg_table()

In [4]:
# FLD removed - not tested in JAR for prio_hol_open
# solver = FLD(model)
# AvgTable = solver.avg_table()

In [5]:
# MVA with seed=23000 to match JAR test
solver = MVA(model, seed=23000, verbose=VerboseLevel.SILENT)
AvgTable = solver.avg_table()

Station JobClass   QLen  Util  RespT  ResidT  ArvR  Tput
 Source   Class1 0.0000  0.00 0.0000  0.0000   0.0   0.1
 Source   Class2 0.0000  0.00 0.0000  0.0000   0.0   0.1
 Source   Class3 0.0000  0.00 0.0000  0.0000   0.0   0.1
 Queue1   Class1 0.1364  0.12 1.3636  1.3636   0.1   0.1
 Queue1   Class2 0.2500  0.20 2.5000  2.5000   0.1   0.1
 Queue1   Class3 0.3158  0.24 3.1579  3.1579   0.1   0.1
 Queue2   Class1 0.1236  0.11 1.2360  1.2360   0.1   0.1
 Queue2   Class2 0.1494  0.13 1.4943  1.4943   0.1   0.1
 Queue2   Class3 0.1765  0.15 1.7647  1.7647   0.1   0.1
 Queue3   Class1 0.2500  0.20 2.5000  2.5000   0.1   0.1
 Queue3   Class2 0.2658  0.21 2.6582  2.6582   0.1   0.1
 Queue3   Class3 0.2346  0.19 2.3457  2.3457   0.1   0.1
 Queue4   Class1 0.3333  0.25 3.3333  3.3333   0.1   0.1
 Queue4   Class2 0.2346  0.19 2.3457  2.3457   0.1   0.1
 Queue4   Class3 0.7544  0.43 7.5439  7.5439   0.1   0.1


In [ ]:
# JMT with seed=23000, samples=10000 to match MATLAB
solver = JMT(model, seed=23000, samples=10000, verbose=VerboseLevel.SILENT)
AvgTable = solver.avg_table()

In [7]:
# SSA with seed=23000, samples=10000 to match JAR test
solver = SSA(model, seed=23000, samples=10000, verbose=VerboseLevel.SILENT)
AvgTable = solver.avg_table()

Station JobClass   QLen   Util   RespT  ResidT   ArvR   Tput
 Source   Class1 0.0000 0.0000  0.0000  0.0000 0.0000 0.1000
 Source   Class2 0.0000 0.0000  0.0000  0.0000 0.0000 0.1000
 Source   Class3 0.0000 0.0000  0.0000  0.0000 0.0000 0.1000
 Queue1   Class1 0.3011 0.1299  0.7048  0.7048 0.4272 0.4272
 Queue1   Class2 0.4409 0.1875  1.1494  1.1494 0.3836 0.3836
 Queue1   Class3 0.5817 0.2512  1.3872  1.3872 0.4193 0.4193
 Queue2   Class1 0.1808 0.1075  1.8182  1.8182 0.0994 0.0994
 Queue2   Class2 0.1992 0.1201  2.1472  2.1472 0.0928 0.0928
 Queue2   Class3 0.2020 0.1378  1.9450  1.9450 0.1038 0.1038
 Queue3   Class1 0.4794 0.2184  4.3099  4.3099 0.1112 0.1112
 Queue3   Class2 0.4952 0.2172  5.1210  5.1210 0.0967 0.0967
 Queue3   Class3 0.4543 0.1968  4.3335  4.3335 0.1048 0.1048
 Queue4   Class1 3.6996 0.3183 33.4856 33.4856 0.1105 0.1105
 Queue4   Class2 1.5591 0.1702 17.1706 17.1706 0.0908 0.0908
 Queue4   Class3 4.4613 0.4295 43.2708 43.2708 0.1031 0.1031
